In [3]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cac


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, 
    chunk_overlap = 100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

loader = Docx2txtLoader('./data/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)
print(len(document_list))
print(f'각 청크 사이즈: {list(len(text.page_content) for text in document_list)}')



1426
각 청크 사이즈: [237, 233, 267, 290, 279, 295, 176, 257, 102, 297, 67, 265, 289, 257, 141, 237, 281, 230, 288, 270, 294, 236, 270, 291, 289, 278, 184, 248, 178, 252, 262, 286, 138, 251, 181, 264, 173, 299, 130, 91, 298, 273, 294, 120, 261, 48, 295, 33, 294, 277, 234, 245, 234, 246, 198, 296, 275, 266, 283, 277, 127, 299, 143, 91, 298, 273, 294, 120, 261, 48, 295, 33, 294, 277, 234, 245, 234, 246, 198, 296, 275, 266, 207, 262, 230, 253, 157, 293, 149, 283, 153, 255, 276, 40, 294, 178, 221, 284, 272, 212, 100, 297, 120, 297, 100, 292, 147, 215, 254, 296, 284, 293, 269, 286, 273, 227, 270, 277, 283, 288, 265, 247, 222, 280, 281, 106, 241, 284, 115, 258, 270, 290, 289, 243, 288, 247, 222, 280, 281, 106, 241, 284, 115, 258, 270, 238, 274, 279, 250, 293, 293, 187, 249, 260, 285, 244, 278, 209, 197, 191, 293, 291, 190, 273, 255, 192, 266, 281, 250, 267, 293, 190, 293, 295, 294, 291, 190, 273, 255, 192, 266, 281, 250, 267, 293, 290, 232, 295, 251, 209, 298, 287, 279, 241, 295, 253, 160, 202, 18

In [7]:
# 임베딩 
from langchain_openai import OpenAIEmbeddings
embdding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:

# # 데이터 처음 저장 할 때
# from langchain_chroma import Chroma
# datavase = Chroma.from_documents(
#     documents=document_list,
#     embedding=embdding,
#     collection_name='chroma_tax',
#     persist_directory='./chroma'
# )

: 

In [2]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(document_list, embdding, index_name=index_name)






In [37]:
query = '연봉 5천만원 직장인 종합소득세는 얼마인가?'
# query = '연봉 5천만원 거주자의 종합소득세는 얼마인가?'

In [42]:
retriever = vectorstore.as_retriever(search_kwargs={'k' : 3})
retriever.invoke(query)

[Document(id='7576f404-f670-462e-961a-741c09facb2d', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='cb0173b2-e260-45b7-8fa2-b1cf18869a36', metadata={'source': './data/tax.docx'}, page_content='| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|'),
 Document(id='d164ccaf-2cd3-4d85-950a-cca205a38246', metadata={'source': './data/tax.docx'}, page_content='3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [33]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=['context', 'questrion'],
#     template="""
# [Identitiy]
# - 당신은 최고의 한국 소득세 전문가 입니다. 
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.

# [Context]
# {context}

# Question: {question}
# """
# )

In [43]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = retriever,
    chain_type_kwargs = {"prompt": prompt},
)

In [44]:
ai_message= qa_chain.invoke({"query": query})

In [45]:
ai_message

{'query': '연봉 5천만원 직장인 종합소득세는 얼마인가?',
 'result': '연봉 5천만원 직장인의 종합소득세는 과세표준 1,400만원 초과 5,000만원 이하 구간에 해당하여, 84만원 + (1,400만원을 초과하는 금액의 15%)입니다. 따라서, 종합소득세는 84만원 + (36,000,000원 x 15%) = 84만원 + 540만원 = 624만원이 됩니다.'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
        사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
        만약 변경할 필요가 없다고 판단된다면, 사용자의 질문으르 변경하지 않아도 됩니다.
                                            사젼 : {dictionary}
        질문: {{question}}                                                                        
""")

dictionary.chain = prompt |llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} 


In [ ]:
nuew_question = dictionary.chain.invok{{"query": query})

In [ ]:
ai_message= qa_chain.invoke({"query": query})